<img src='https://github.com/LinkedEarth/Logos/raw/master/PYLEOCLIM_logo_HORZ-01.png' width="800">


# Working with `Pandas`

by [Julien Emile-Geay](https://orcid.org/0000-0001-5920-4751)

[Pandas](https://pandas.pydata.org) is part of nearly every data scientist's toolkit, with robust support for spreadsheet-like data structures. Until 2023, however, this workhorse of time series analysis was unusable in the paleogeosciences, because Pandas long ago hardcoded nanoseconds as the base unit of time. This limited the timescales it can represent on a 64-bit machine to a relatively narrow timespan of 585 years, thus excluding many paleogeoscience applications (for more explanations, see [this blog post](https://medium.com/cyberpaleo/pandas-and-the-geosciences-a-4-5-billion-year-story-66af9f565a4b)). In this notebook we explore the synergies between pandas and Pyleoclim.  

In [ ]:
%load_ext watermark
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pyleoclim as pyleo

Let us load the SOI timeseries:

In [ ]:
ts = pyleo.utils.load_dataset('SOI')
ts.plot(invert_yaxis=True) # invert y axis so El Niño events plot upward

There are two properties attached to `pyleo.Series` objects:
1. a Pandas [datetime_index](https://pandas.pydata.org/docs/dev/reference/api/pandas.DatetimeIndex.html):

In [ ]:
ts.datetime_index

2. a dictionary bundling all the metadata:

In [ ]:
ts.metadata

Invoking the object itself returns some essential metadata and a compressed view of the data.

In [ ]:
ts

For a prettier display (in Jupyter notebook only):

In [ ]:
ts.view()

## .to_pandas()

It is easy to export a Pyleoclim Series to a Pandas Series:

In [ ]:
pdts = ts.to_pandas() #  returns just the Series ; metadata are available at ts.metadata
type(pdts) 

It is now a bona fide Pandas Series, and we can do with it everything we might do with Pandas. For example:

In [ ]:
pdts.head()

Or this:

In [ ]:
pdts.describe()

Because Pyleoclim now has Pandas under the hood, one can now apply any Pandas method to a Pyleoclim Series, via a [lambda function](https://www.freecodecamp.org/news/python-lambda-function-explained/). For instance, applying an exponential transform to the data:

In [ ]:
ts.pandas_method(lambda x: x.transform(np.exp))

For more examples, see [this page](https://sparkbyexamples.com/pandas/pandas-apply-with-lambda-examples/?utm_content=expand_article)

## Unit conversions

Pyleoclim now comprehends datestring semantics, which enable enhanced conversions betwen time representations. For instance, converting the SOI series to years before 1950 ("BP"): 

In [ ]:
tsBP = ts.convert_time_unit('yrs BP')  
fig, ax = tsBP.plot(invert_yaxis=True) # by default, plots represent values in increasing order, so we reverse the x-axis

The Series plots from recent to old, because the Matplotlib `plot()` function always works with increasing values. However, it is easy to check that it was indeed flipped:

In [ ]:
tsBP.view()

The index has been flipped as well:

In [ ]:
tsBP.datetime_index

If we wanted to preserve the original time flow (old to recent) in a plot, all you'd have to do is use the `invert_xaxis` parameter:

In [ ]:
tsBP.plot(invert_yaxis=True, invert_xaxis=True, label = 'SOI, years BP', color='C1') 

(this is an admittedly contrived example, as no one in their right mind would cast instrumental series in years BP). 

## Selection

Let's load a true paleoclimate example, the Dome C $CO_2$ [composite](https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/antarctica2015co2composite.txt), and use Pandas semantics to select particular values.

In [ ]:
co2ts = pyleo.utils.load_dataset('AACO2')
co2ts.plot(color='gray')

In [ ]:
co2ts.metadata

In [ ]:
co2ts.view()

To select a particular interval, (say the [Eemian](https://en.wikipedia.org/wiki/Eemian), 115 to 130 ky BP), you can use `sel`:

In [ ]:
co2_ee = co2ts.sel(time=slice(115,130))
co2_ee.plot(marker='o', color='gray')

If you wanted to extract the time of the maximum value, an easy approach is to use `sel` again:

In [ ]:
co2_ee.sel(value=co2_ee.value.max()).view()

We zero-ed in on the value ~128.5 ky BP as the highest $CO_2$ concentration in the Eemian. Note that the result is still a `Series` object (in this case, a very short one!).  Similarly, to identify the timing of glacial maxima by values below 200 ppm, one could select with an open interval:

In [ ]:
co2_G = co2ts.sel(value=slice(None,200)) # returns a new Series object
fig, ax = co2ts.plot(color='gray')
co2_G.plot(marker='o',linewidth=0,alpha= 0.5, ax=ax,label=r"$CO_2\leq200$") 

For interglacials, select half-open intervals the other way:

In [ ]:
co2_IG = co2ts.sel(value=slice(270,None)) # returns a new Series object
fig, ax = co2ts.plot(color='gray')
co2_G.plot(marker='o',linewidth=0,alpha= 0.5, ax=ax,label=r"$CO_2\leq200$") 
co2_IG.plot(marker='o',linewidth=0, color = 'C3', alpha= 0.5, ax=ax,label=r"$CO_2\geq270$") 

Again, both of these are `Series` objects, albeit highly discontinuous ones. 

Science Remark: admittedly, an absolute threshold for defining interglacials makes little sense. A smarter way would be to use [clustering](https://en.wikipedia.org/wiki/Cluster_analysis), as implemented in the `outliers()` method and a [dedicated tutorial](./L2_outlier_detection.md). 

## CSV Import/Export

Pandas integration allows a very easy round trip with CSV files:

### Exporting to CSV

In [ ]:
filename = '../data/EPICA_Dome_C_CO2.csv'
co2ts.to_csv(path = filename)

The path name is optional; if no file name is provided (as is the case here), the file is named after the `Series` label. This is another reason to choose meaningful and relatively concise labels!

### Importing from CSV
Read the file back in and gives the same `Series`:

In [ ]:
co2ts2 = pyleo.Series.from_csv(path='../data/EPICA_Dome_C_CO2.csv')
co2ts2.equals(co2ts) 

Note the use of the `equals()` method, inspired the [eponymous method from Pandas](https://pandas.pydata.org/docs/reference/api/pandas.Series.equals.html). The Pyleoclim implementation returns two boolean objects: the first says whether the two Pandas `Series` are the same; the second whether the metadata are the same. Fortunately, we landed back on our feet. 

## Resampling

One of Pandas' most powerful capabilities is [resampling()](https://pandas.pydata.org/docs/user_guide/timeseries.html#resampling) series to attain various temporal resolutions. Pyleoclim implements a function of the same name, but using normal "paleo-speak" to define resolution. For instance, let's coarse-grain on 5,000 year intervals:

In [ ]:
co2_5k = co2ts.resample('5ka')
type(co2_5k)

The output of this function is a variant on a Pandas [resampler](https://pandas.pydata.org/docs/reference/resampling.html); the values then need to be aggregated or transformed. For our purpose, let's average them over those 5ky bins:

In [ ]:
co2_5kavg = co2_5k.mean() # the aggregator here is simply the mean
fig, ax = co2ts.plot(color='gray')
co2_5kavg.plot(ax=ax,color='C1')         

One notable distinction to standard Pandas resampling is that Pyleoclim aligns the resampled index to the _midpoint_ of each interval, to minimize age offsets. In contrast, Pandas aligns to the beginning of an interval.  

In terms of nomenclature, `resample()` can understand several abbreviations for "kiloyear", such as 'ka', 'ky', or 'kyrs'. In fact, these would work:

In [ ]:
pyleo.utils.tsbase.MATCH_KA

For millions of years, use:

In [ ]:
pyleo.utils.tsbase.MATCH_MA

and so on for other multiples, like years (`pyleo.utils.tsbase.MATCH_A`) or billion years (`pyleo.utils.tsbase.MATCH_GA`) 


Further, the `Resampler` class allows one to choose statistics other than the mean, e.g., a running standard deviation:

In [ ]:
co2_5k.std().view()

## Pandas operations with MultipleSeries objects

Let's load the Deuterium EPICA DOME C record:

In [ ]:
edc = pyleo.utils.load_dataset('EDC-dD')

We then create a `MultipleSeries` object using the `&` shorthand:

In [ ]:
ms =  edc.convert_time_unit("ky BP") & co2ts
type(ms)

In [ ]:
ms.stackplot()

### Exporting to pandas

The `MultipleSeries` class also has a `to_pandas()` method; this one exports to a dataframe:

In [ ]:
df = ms.to_pandas()
df.head()

By default, `to_pandas()` exports the `Series` as they are, and one can see that the values are not aligned. Some plotting functions still work out of the proverbial box:

In [ ]:
df.hist()

Alas, `df.plot()` wouldn't work, as Matplotlib assumes dates must be between year 0001 and 9999. 

As with `Series`, `ms.to_csv()` will export the object to a CSV file. We skip this here, as the output is not very graphical. 

### Aligning to a common axis
In many instances it is useful for the `Series` to share a common time axis. This can be done simply this way:

In [ ]:
df = ms.to_pandas(use_common_time=True)
df.head()

Seaborn is a plotting library designed to visualize DataFrames. With just one line, it is easy to get an overview of the data with a [`pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html):

In [ ]:
sns.set(font_scale=0.8)
with plt.style.context('bmh'):
    sns.pairplot(df)  

or a [`jointplot`](https://seaborn.pydata.org/generated/seaborn.jointplot.html?highlight=jointplot#seaborn.jointplot):

In [ ]:
with plt.style.context('bmh'):
    sns.jointplot(data=df,kind='kde', x='EPICA Dome C dD', y='EPICA Dome C CO2')  

## Summary
The legendary pandas library is now fully integrated within Pyleoclim. This opens the door to many powerful, user-friendly functionatlities for data processing and visualization. For other examples of how pandas is making life easier in Pyleoclim, see [this notebook](L0_basic_ts_manipulation.md); however, the real frontier is what you will decide to do with it. Possibilities abound!

In [ ]:
%watermark -n -u -v -iv -w